In [77]:
# 문제를 제대로 이해하고 하기
# 문제 솔직히 대충 이해해서 인지 코드에서 왜 그렇게 나오는 지에 대해 이해안가는 부분이 있었다
# 왜 꼭 두번째 시도에 문제를 제대로 이해 ? -> 근데 문제를 보지 않아서인지 제대로 이해 하지 못했어(metric부분)이건  천재의 영역 아냐 ? #아니 천재도 알 수 없다 그건 대회의 측정 기준이 따로 있다면; 천재가 되려할 필요도 없다
# 문제 제대로 이해해야 좀 더 막막하지 않고 자신감이 생긴다

In [78]:
# 글쓰기는 성공을 위한 중요한 기술 - 배경  , 필요성 , 목적
# 그러나 고등학생 3분의 1 ,  저소득층,흑인,히스패닉은 쓰기 능력이 15% 미만으로 훨씬 더 나쁘다 - 문제
# 많은 사람이 에세이에서 논문 진술 , 주장 지원 등 쓰기 구조를 식별하지 못하거나 철저히 식별하지 않는다 - 문제
# 이러한 문제는 독립적으로 백업이안되는 알고리즘이 포함된 독점 도구, 많은 비용 - 문제
# 유색 인종 , 저소득 배경의 학생을에게 서비스 미제공 학교의 경우 악화 <-문제
# 자동화된 쓰기 피드백 분야는 교육 민주화에 도움이 될 수 있는 혁신 <-목적
# 자동화된 쓰기 피드백 제공 - 해결 방안
# 요약하되 너무 요약하지 않도록, 미니멀리즘과 맥시멀리즘 x

In [79]:
# 정답과 예측 간의 겹침 >= 0.5 , 예측과 정답 간의 겹침 >=0.5 -> 예측 일치


In [80]:
DOWNLOADED_MODEL_PATH = '../input/tf-longformer-v12'

In [81]:
import os

import pandas as pd , numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# from transformers import *
from transformers import AutoTokenizer, AutoConfig, TFAutoModel

print("TF version", tf.__version__)


TF version 2.8.0


In [82]:
import os

In [83]:
os.environ["CUDA_VISIBLE_DEVICES"]

KeyError: 'CUDA_VISIBLE_DEVICES'

In [ ]:
# os 를 먼저 검사하고 strategy 를 정한다
# USE MULTIPLE GPUS
if os.environ["CUDA_VISIBLE_DEVICES"].count(',') == 0:
    strategy = tf.distribute.get_strategy()
    print('single strategy')
else:
    strategy = tf.distribute.MirroredStrategy()
    print("multiple strategy")


In [84]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision" : True})

In [85]:
print("Mixed precision enabled")

Mixed precision enabled


In [86]:
train = pd.read_csv('../input/feedback-prize-2021/train.csv')

In [87]:
print(train.shape)
train.head()

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [88]:
print('The train labels are:')
train.discourse_type.unique()

The train labels are:


array(['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement',
       'Counterclaim', 'Rebuttal'], dtype=object)

In [89]:
IDS = train.id.unique()
print('There are', len(IDS), 'train texts')

There are 15594 train texts


## Tokenize Train
The following code converts Kaggle's train dataset into a NER Token array that we can use to train a NER transformer. i have made it very clear which targets belong to which class. This allows us to very easily convert this code to Question Answer formulation if we want.Just change the 14 NER arrays to be 14 arrays of  start position and end position for each of the 7 clasees.(You will need to think creatively
what to do if a singletext has multiple of one class

In [90]:
MAX_LEN = 1024

# THE TOKENS AND ATTENTION ARRAYS

tokenizer = AutoTokenizer.from_pretrained(DOWNLOADED_MODEL_PATH)
train_tokens = np.zeros((len(IDS), MAX_LEN) , dtype='int32')
train_attention = np.zeros((len(IDS), MAX_LEN), dtype = 'int32')


# THE 14 CLASSES FOR NER
lead_b = np.zeros((len(IDS),MAX_LEN))
lead_i = np.zeros((len(IDS),MAX_LEN))

position_b = np.zeros((len(IDS),MAX_LEN))
position_i = np.zeros((len(IDS),MAX_LEN))

evidence_b = np.zeros((len(IDS),MAX_LEN))
evidence_i = np.zeros((len(IDS),MAX_LEN))

claim_b = np.zeros((len(IDS),MAX_LEN))
claim_i = np.zeros((len(IDS),MAX_LEN))

conclusion_b = np.zeros((len(IDS),MAX_LEN))
conclusion_i = np.zeros((len(IDS),MAX_LEN))

counterclaim_b = np.zeros((len(IDS),MAX_LEN))
counterclaim_i = np.zeros((len(IDS),MAX_LEN))

rebuttal_b = np.zeros((len(IDS),MAX_LEN))
rebuttal_i = np.zeros((len(IDS),MAX_LEN))

# HELPER VARIABLES
train_lens = []
targets_b = [lead_b, position_b, evidence_b, claim_b, conclusion_b, counterclaim_b, rebuttal_b]
targets_i = [lead_i, position_i, evidence_i, claim_i, conclusion_i, counterclaim_i, rebuttal_i]
target_map = {'Lead':0, 'Position':1, 'Evidence':2, 'Claim':3, 'Concluding Statement':4,
             'Counterclaim':5, 'Rebuttal':6}

Didn't find file ../input/tf-longformer-v12/added_tokens.json. We won't load it.
loading file ../input/tf-longformer-v12/vocab.json
loading file ../input/tf-longformer-v12/merges.txt
loading file ../input/tf-longformer-v12/tokenizer.json
loading file None
loading file ../input/tf-longformer-v12/special_tokens_map.json
loading file ../input/tf-longformer-v12/tokenizer_config.json


In [91]:
# WE ASSUME DATAFRAME IS ASCENDING WHICH IT IS
# True 가 아닐 시 AssertError 발생
# 좋은 걸 인지시키려면 반복하자
assert( np.sum(train.groupby('id')['discourse_start'].diff() <=0) ==0)

In [92]:
%reload_ext autoreload

In [93]:
%autoreload 2

In [94]:
LOAD_TOKENS_FROM = '../input/tf-longformer-v12'

In [95]:
# FOR LOOP THROUGH EACH TRAIN TEXT
for id_num in range(len(IDS)):
    #if LOAD_TOKENS_FROM: break
    if id_num%100 == 0: print(id_num, ', ', end='')

    # READ TRAIN TEXT , TOKENIZE , AND SAVE IN TOKEN ARRAYS
    n = IDS[id_num]
    name = f'../input/feedback-prize-2021/train/{n}.txt'
    txt = open(name, 'r').read()
    train_lens.append(len(txt.split()))

    tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN , padding='max_length',
                                      truncation= True , return_offsets_mapping=True)

    train_tokens[id_num, ] = tokens['input_ids']
    train_attention[id_num,] = tokens['attention_mask']

    # FIND TARGETS IN TEXT AND SAVE IN TARGET ARRAYS
    offsets = tokens['offset_mapping']
    offset_index = 0
    df = train.loc[train.id==n]
    for index,row in df.iterrows():
        a = row.discourse_start
        b = row.discourse_end
        if offset_index>len(offsets)-1:
            break
        c = offsets[offset_index][0]
        d = offsets[offset_index][1]

        beginning = True

        while b>c:
            if (c>=a)&(b>=d):
                k = target_map[row.discourse_type]
                if beginning:
                    targets_b[k][id_num][offset_index] = 1
                    beginning = False
                else:
                    targets_i[k][id_num][offset_index] = 1
            offset_index += 1
            if offset_index > len(offsets) -1:
                break
            c = offsets[offset_index][0]
            d = offsets[offset_index][1]

0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 12500 , 12600 , 12700 , 12800 , 12900 , 13000 , 13100 , 13200 , 13300 , 13400 , 13500 , 13600 , 13700 , 13800 , 

In [96]:
train_tokens

array([[    0, 48083, 50118, ...,     1,     1,     1],
       [    0, 48083,   359, ...,     1,     1,     1],
       [    0, 40216, 12091, ...,     1,     1,     1],
       ...,
       [    0, 31206,  1818, ...,     1,     1,     1],
       [    0,   970,    34, ...,     1,     1,     1],
       [    0,  1121,  8178, ...,   768,     6,     2]], dtype=int32)

In [97]:
targets[0]

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int32)

In [98]:
targets = np.zeros((len(IDS), MAX_LEN, 15), dtype='int32')

for k in range(7):
    targets[:,:,2*k] = targets_b[k]
    targets[:,:,2*k+1] = targets_i[k]
targets[:,:,14] = 1-np.max(targets,axis=-1)

In [103]:
def build_model():
    tokens = tf.keras.layers.Input(shape=(MAX_LEN, ), name='tokens', dtype=tf.int32)
    attention = tf.keras.layers.Input(shape=(MAX_LEN,), name ='attention', dtype=tf.int32)

    config = AutoConfig.from_pretrained(DOWNLOADED_MODEL_PATH+'/config.json')
    backbone = TFAutoModel.from_pretrained(DOWNLOADED_MODEL_PATH+'/tf_model.h5', config=config)

    x = backbone(tokens, attention_mask=attention)
    x = tf.keras.layers.Dense(256, activation='relu')(x[0])
    x = tf.keras.layers.Dense(15,  activation='softmax', dtype='float32')(x)


    model = tf.keras.Model(inputs=[tokens, attention] , outputs=x)
    model.compile(optimizer= tf.keras.optimizers.Adam(lr = 1e-4),
                   loss = [tf.keras.losses.CategoricalCrossentropy()],
                   metrics = [tf.keras.metrics.CategoricalAccuracy()])
    return model

In [104]:
with strategy.scope():
    model = build_model()

loading configuration file ../input/tf-longformer-v12/config.json
Model config LongformerConfig {
  "_name_or_path": "../input/tf-longformer-v12/config.json",
  "architectures": [
    "LongformerModel"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sep_token_id": 2,
  "transformers_version": "4.16.2",
  "type_vocab_size": 1,
  "use_cache": tru

In [105]:
model

In [106]:
# Learning rate schedule and model checkpoint
EPOCHS = 5
BATCH_SIZE = 4
LRS = [0.25e-4, 0.25e-4, 0.25e-4, 0.25e-4, 0.25e-5]

In [107]:
EPOCHS

5

In [108]:
def lrfn(epoch):
    return LRS[epoch]


lr_callback  = tf.keras.callbacks.LearningRateScheduler(lrfn , verbose=True)

In [109]:
# TRAIN VALID SPLIT 90% 10%
np.random.seed(42)
train_idx = np.random.choice(np.arange(len(IDS)), int(0.9*len(IDS)), replace=False)
valid_idx = np.setdiff1d(np.arange(len(IDS)), train_idx)
np.random.seed(None)
print("Train size", len(train_idx), ', Valid size',len(valid_idx))

Train size 14034 , Valid size 1560


In [110]:
model.fit(x = [train_tokens[train_idx,], train_attention[train_idx,]] ,
           y= targets[train_idx,],
            validation_data = ([train_tokens[valid_idx,], train_attention[valid_idx,]],
                                 targets[valid_idx,]),
          callbacks= [lr_callback],
          epochs= EPOCHS,
          batch_size = BATCH_SIZE,
          verbose = 2)
# SAVE MODEL WEIGHTS
model.save_weights(f'long_v{VER}.h5')


Epoch 1: LearningRateScheduler setting learning rate to 2.5e-05.
Epoch 1/5


KeyboardInterrupt: 

In [ ]:


# prediction 은 class에서 원래 string값으로 변환이 되고 target_map_rev[int(start)] 부분
# offsetmapping 을통해 Predictionstring 을 비교한다
#id/class/predictionstring
# 대회 평가 metric에 의해
# 예측한 string들이 validation의 predictionstring과 0.5이상 겹친다면 potential_true positive 이다
#  np.argmax를 취했으므로 argmax에 해당하는 string 만 뽑게 되므로
#  word_map =  -1 * np.ones
#  if off[i][1] == 0: continue
#  prediction.append(word_map[i])
# while pred[i] == start+0.5: (start:0,1,2,3,4,5,6,7) 0~0.5 same, 1~1.5 same class..
#  if not word_map[i] in prediction:
#
#  [x for x in prediction if x != -1] word_map에서 핵심 줄기코드
#  if prediction>4: -> df  일종의 psudo code 먼저 이렇게 작성하는 것도 괜찮을 듯 하다
#  validation과는 안 맞을 수 있지만
#핵심 :   같은 클래스로 분류된 값들을 prediction 에 올린다 짧게 보지 말고 길게 좀 보자 , 그게 코드 읽는법 , 단어바이단어,
# 라인바이라인으로만 읽지말고 , 함수단위 모듈단위 , 자꾸 관성으로 행동하지말것
# 자유자재로 왔다갔다 해야한다 워드바이워드, 라인바이라인, 펑션바이펑션, 모듈바이모듈 , 그러면서 한가지 방법으로만 보면 안되고 multiuse필요
# -> 코드 if start in [0,1,2,3,4,5,6,7]:
#           prediction.append(word_map[i])  word position in chars(공백등이 포함된)
#           i += 1
#           if i >= MAX_LEN:break # 해당 id 길이를 전부 순회했다면 break
#           전체를 이해해야 이해되는 한 라인이 있는데 이 아래 while 라인이  그렇다
#           while pred[i]==start+0.5: 이 라인이 모든 걸 설명 예측한 모델의 output의 pred i번째가 계속 start class와 같은
#               (+0.5) because targets_b(egin) target_i(intermideate?) 같은 클래스지만 시작과 중간을 나누었었기때문
#           그리고 위에서 pred 를 2로 나누었으므로 , while 조건 :에서 조건 자체가  i번째가 start와 연속적으로 같은 class라는 것을 의미한다
#
# 제대로 학습된 모델이라면 쓸만한 모델이다


In [63]:
x = np.array([[4,2,3],[1,0,3]])
index_array = np.argmax(x,axis=-1) # 마지막 차원의 최댓값 인덱스

In [64]:
index_array

array([0, 2])

In [ ]:
#문서의 id기준으로 tokenizing 을 하고 15개의 클래스 레이블로 학습을 한다
#1024개의 id token이 들어오는 것은 같지만 1024개의 input id들이 전부 클래스 값을 갖도록 학습된다
#따라서 validation 을 할때 해당 id의 token들의 class비율을 가지고
# predicion의 predictionstring이  validation set의 prediction string 이
# 겹치는 비율이 0.5이상이면 해당 클래스라고 인정한다
#

> ## <span style="background-color:rgb(255,0,0);">'스스로 앎'으로 인해서 좋은 것을 반복할 수 있게 하는 자존감이 곧 자신감이 된다</span>